In [ ]:
!pip install pulp

# Round 1 and 2

In [ ]:
import numpy as np
from pulp import *

def calc_sv_zero(sv):
  c = 0
  for i in sv:
    if 0 - 2e-5 <= i <= 0 + 2e-5:
      c += 1
  return c

def calc_fd_zero(fd):
  c = 0
  for i in range(column):
    if flux_d[i] == 0:
      c += 1
  return c

with open('/content/drive/MyDrive/data/E_Coli/modelround1and2.txt', 'r') as f:
  data = f.read()
lines = data.splitlines()
row = int(lines[1].split(',')[1])
column = int(lines[1].split(',')[2])
s = np.zeros((row, column))
up = np.zeros(column)
low = np.zeros(column)
u_starting_index = 0
l_starting_index = 0
flux_d = np.zeros(column)
c = np.ones(column)

for l in range(3, len(lines)):
  try:
    i = int(float(lines[l].split(',')[0])) - 1
    j = int(float(lines[l].split(',')[1])) - 1
    v = float(lines[l].split(',')[2])
    s[i][j] = v
  except ValueError:
    u_starting_index = l + 3
    break
for l in range(u_starting_index, len(lines)):
  try:
    i = int(float(lines[l].split(',')[0])) - 1
    v = float(lines[l].split(',')[1])
    up[i] = v
  except ValueError:
    l_starting_index = l + 3
    break
for l in range(l_starting_index, len(lines)):
  i = int(float(lines[l].split(',')[0])) - 1
  v = float(lines[l].split(',')[1])
  low[i] = v

v = [LpVariable('v' + str(i), lowBound = low[i], upBound = up[i]) for i in range(column) ]
prob = LpProblem("FBA", LpMinimize)
c = []
for i in range(row):
  temp = []
  for j in range(column):
    if s[i][j] != 0:
      temp.append((v[j], s[i][j]))
  c.append(temp)
for i in c:
  prob += LpConstraint(LpAffineExpression(i), LpConstraintEQ, rhs=0)

prob.solve()
for i in range(column):
  flux_d[i] = value(v[i])

sv = np.dot(s, flux_d)

In [ ]:
c = 0
for i in sv:
  if 0 - 2e-5 <= i <= 0 + 2e-5:
    c += 1
print('SV zero count:', c)
print('SV not zero count:', len(sv) - c)
c = 0
for i in range(column):
  if flux_d[i] == 0:
    c += 1
print('Flux.D. zeros:', c)
print('Flux.D. not zeros:', column - c)
print('columns', column)
c = 0
for i, e in enumerate(flux_d):
  if low[i] - 2e-5 <= e <= up[i] + 2e-5:
    continue
  else:
    c += 1
    print("ERROR:", i, low[i], '|', e, '|', up[i])
print('Num of Errors:', c)

In [ ]:
with open('/content/drive/MyDrive/data/answer/output.txt', mode='w') as f:
  for i in flux_d:
    f.write(str(i + 0.0) + '\n')